# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{50844634909: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #tensors:1>

In [14]:
x.id_at_location

50844634909

In [15]:
x.id

85673375777

In [16]:
x.owner

<VirtualWorker id:me #tensors:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:85673375777 -> bob:50844634909]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [3]:
import syft as sy
import torch as th

In [4]:
hook = sy.TorchHook(th)

In [7]:
bob = sy.VirtualWorker(hook, id="bob")

In [5]:
alice = sy.VirtualWorker(hook, id='alice')

In [6]:
x = th.tensor([1,2,3,4,5])

In [10]:
x_ptr = x.send(bob, alice)

In [11]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:55571497257 -> bob:62526561383]
	-> (Wrapper)>[PointerTensor | me:72516695972 -> alice:35777687046]

In [13]:
x_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:55571497257 -> bob:62526561383], 'alice': (Wrapper)>[PointerTensor | me:72516695972 -> alice:35777687046]}

In [14]:
x_ptr.child.child

{'bob': (Wrapper)>[PointerTensor | me:55571497257 -> bob:62526561383],
 'alice': (Wrapper)>[PointerTensor | me:72516695972 -> alice:35777687046]}

In [15]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [20]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [21]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [22]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [23]:
x

(Wrapper)>[PointerTensor | me:4890474838 -> bob:83321263082]

In [24]:
y

(Wrapper)>[PointerTensor | me:16247076050 -> bob:4386425208]

In [25]:
z = x + y

In [26]:
z

(Wrapper)>[PointerTensor | me:76264168488 -> bob:5073220984]

In [27]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [28]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:37789301158 -> bob:8615129030]

In [29]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [36]:
x = th.tensor([15.,12,4,6,78], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [37]:
z = (x + y).sum()

In [38]:
z.backward()

# will create a gradients on x and y

(Wrapper)>[PointerTensor | me:80927284369 -> bob:91698811163]

In [39]:
x = x.get()

In [40]:
x

tensor([15., 12.,  4.,  6., 78.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [45]:
input = th.tensor([[1.,1.],[0,1.],[1.,0.],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]],requires_grad=True).send(bob)

In [46]:
input

(Wrapper)>[PointerTensor | me:66072760891 -> bob:40609239272]

In [47]:
target

(Wrapper)>[PointerTensor | me:20474923531 -> bob:15256398416]

In [51]:
weights = th.tensor([[0.],[0]],requires_grad=True).send(bob)

In [52]:
weights

(Wrapper)>[PointerTensor | me:14479405166 -> bob:30161329549]

In [53]:
pred = input.mm(weights)

In [55]:
loss = ((pred-target)**2).sum()

In [56]:
loss.backward()

(Wrapper)>[PointerTensor | me:13517138294 -> bob:82759403935]

In [57]:
weights.data.sub_(weights.grad*0.1)

(Wrapper)>[PointerTensor | me:14479405166 -> bob:30161329549]::data

In [58]:
weights.grad += 0

In [59]:
print(loss.get().data)

tensor(2.)


In [70]:
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad += 0
    
    print(loss.get().data)

tensor(1.7610)
tensor(0.3246)
tensor(0.5000)
tensor(1.6936)
tensor(1.4617)
tensor(0.5022)
tensor(1.0601)
tensor(2.1290)
tensor(1.4042)
tensor(0.0785)


# Lesson: Garbage Collection and Common Errors


In [46]:
import syft as sy
import torch as th

hook = sy.TorchHook(th)

bob = sy.VirtualWorker(hook, id="bob")

W0730 12:15:25.608108 4576925120 hook.py:98] Torch was already hooked... skipping hooking process


In [47]:
# bob = bob.clear_objects() 
# 1

In [48]:
bob._objects

{96705801607: tensor([1, 2, 3, 4, 5]),
 15170376243: tensor([1, 2, 3, 4, 5]),
 47571811020: tensor([1, 2, 3, 4, 5])}

In [49]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [50]:
bob._objects

{96705801607: tensor([1, 2, 3, 4, 5]),
 15170376243: tensor([1, 2, 3, 4, 5]),
 70760386930: tensor([1, 2, 3, 4, 5])}

In [11]:
x.child.garbage_collect_data

True

In [12]:
del x

# 2

In [13]:
bob._objects

{}

In [14]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [15]:
x

(Wrapper)>[PointerTensor | me:83828165431 -> bob:96705801607]

In [16]:
bob._objects

{}

In [17]:
x = "asdf"

# 3

In [18]:
bob._objects

{}

In [19]:
# bob = bob.clear_objects() 
# bob._objects

In [51]:
Out



{4: {},
 6: {},
 8: {},
 10: {},
 11: True,
 13: {},
 15: (Wrapper)>[PointerTensor | me:83828165431 -> bob:96705801607],
 16: {},
 18: {},
 22: (Wrapper)>[PointerTensor | me:53715007093 -> bob:15170376243],
 23: {},
 25: {},
 28: {96705801607: tensor([1, 2, 3, 4, 5]),
  15170376243: tensor([1, 2, 3, 4, 5]),
  70760386930: tensor([1, 2, 3, 4, 5])},
 30: {96705801607: tensor([1, 2, 3, 4, 5]),
  15170376243: tensor([1, 2, 3, 4, 5]),
  70760386930: tensor([1, 2, 3, 4, 5])},
 33: {},
 35: {},
 38: {96705801607: tensor([1, 2, 3, 4, 5]),
  15170376243: tensor([1, 2, 3, 4, 5]),
  70760386930: tensor([1, 2, 3, 4, 5])},
 40: {96705801607: tensor([1, 2, 3, 4, 5]),
  15170376243: tensor([1, 2, 3, 4, 5]),
  70760386930: tensor([1, 2, 3, 4, 5])},
 43: {},
 45: {},
 48: {96705801607: tensor([1, 2, 3, 4, 5]),
  15170376243: tensor([1, 2, 3, 4, 5]),
  70760386930: tensor([1, 2, 3, 4, 5])},
 50: {96705801607: tensor([1, 2, 3, 4, 5]),
  15170376243: tensor([1, 2, 3, 4, 5]),
  70760386930: tensor([1, 2, 3

In [21]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [22]:
x

(Wrapper)>[PointerTensor | me:53715007093 -> bob:15170376243]

In [23]:
bob._objects

{}

In [24]:
x = "asdf"

In [25]:
bob._objects

{}

In [224]:
del x

In [225]:
bob._objects

{98686728660: tensor([1, 2, 3, 4, 5])}

In [253]:
bob = bob.clear_objects()
bob._objects

{}

In [254]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)
    print(bob._objects)
    if i%200==0:
        x = "x updated myself!"
        print(str(i)+ " : "+ x)
        print(bob._objects)

{90066733019: tensor([1, 2, 3, 4, 5])}
0 : x updated myself!
{}
{83786967639: tensor([1, 2, 3, 4, 5])}
{60027067107: tensor([1, 2, 3, 4, 5])}
{76099068231: tensor([1, 2, 3, 4, 5])}
{63373986908: tensor([1, 2, 3, 4, 5])}
{86046428865: tensor([1, 2, 3, 4, 5])}
{71105863156: tensor([1, 2, 3, 4, 5])}
{35204441025: tensor([1, 2, 3, 4, 5])}
{16392833493: tensor([1, 2, 3, 4, 5])}
{91605909702: tensor([1, 2, 3, 4, 5])}
{58305994811: tensor([1, 2, 3, 4, 5])}
{74326611542: tensor([1, 2, 3, 4, 5])}
{92543127730: tensor([1, 2, 3, 4, 5])}
{93638655491: tensor([1, 2, 3, 4, 5])}
{32245034984: tensor([1, 2, 3, 4, 5])}
{38207538784: tensor([1, 2, 3, 4, 5])}
{68297187340: tensor([1, 2, 3, 4, 5])}
{56259294121: tensor([1, 2, 3, 4, 5])}
{19549107446: tensor([1, 2, 3, 4, 5])}
{24798290102: tensor([1, 2, 3, 4, 5])}
{71277056632: tensor([1, 2, 3, 4, 5])}
{82480297346: tensor([1, 2, 3, 4, 5])}
{55550775285: tensor([1, 2, 3, 4, 5])}
{91908097446: tensor([1, 2, 3, 4, 5])}
{16721452437: tensor([1, 2, 3, 4, 5])}


{50183130294: tensor([1, 2, 3, 4, 5])}
{15767430022: tensor([1, 2, 3, 4, 5])}
{70965695274: tensor([1, 2, 3, 4, 5])}
{1179172994: tensor([1, 2, 3, 4, 5])}
{71858667143: tensor([1, 2, 3, 4, 5])}
{6028889988: tensor([1, 2, 3, 4, 5])}
{40829927674: tensor([1, 2, 3, 4, 5])}
{92423231676: tensor([1, 2, 3, 4, 5])}
{94795444043: tensor([1, 2, 3, 4, 5])}
{9073438565: tensor([1, 2, 3, 4, 5])}
{42323842932: tensor([1, 2, 3, 4, 5])}
{66426543399: tensor([1, 2, 3, 4, 5])}
{39508716131: tensor([1, 2, 3, 4, 5])}
{15259732381: tensor([1, 2, 3, 4, 5])}
{98754051116: tensor([1, 2, 3, 4, 5])}
{45765043709: tensor([1, 2, 3, 4, 5])}
{90081836262: tensor([1, 2, 3, 4, 5])}
{66224596501: tensor([1, 2, 3, 4, 5])}
{62785542356: tensor([1, 2, 3, 4, 5])}
{43351984845: tensor([1, 2, 3, 4, 5])}
{34039784642: tensor([1, 2, 3, 4, 5])}
{79517405908: tensor([1, 2, 3, 4, 5])}
{11161247514: tensor([1, 2, 3, 4, 5])}
{89266712954: tensor([1, 2, 3, 4, 5])}
{38530251066: tensor([1, 2, 3, 4, 5])}
{71219254359: tensor([1, 2, 

{6566140717: tensor([1, 2, 3, 4, 5])}
{59577035617: tensor([1, 2, 3, 4, 5])}
{24514920155: tensor([1, 2, 3, 4, 5])}
{77671201364: tensor([1, 2, 3, 4, 5])}
{63627874859: tensor([1, 2, 3, 4, 5])}
{80462923487: tensor([1, 2, 3, 4, 5])}
{3002513135: tensor([1, 2, 3, 4, 5])}
{98648284005: tensor([1, 2, 3, 4, 5])}
{55742027812: tensor([1, 2, 3, 4, 5])}
{49941135318: tensor([1, 2, 3, 4, 5])}
{65035886543: tensor([1, 2, 3, 4, 5])}
{82988095233: tensor([1, 2, 3, 4, 5])}
{14416420403: tensor([1, 2, 3, 4, 5])}
{31630811795: tensor([1, 2, 3, 4, 5])}
{65415825022: tensor([1, 2, 3, 4, 5])}
{30428564918: tensor([1, 2, 3, 4, 5])}
{59468503175: tensor([1, 2, 3, 4, 5])}
{34242950208: tensor([1, 2, 3, 4, 5])}
{48479091349: tensor([1, 2, 3, 4, 5])}
{43897294751: tensor([1, 2, 3, 4, 5])}
{93663803629: tensor([1, 2, 3, 4, 5])}
{88300748317: tensor([1, 2, 3, 4, 5])}
{65570157331: tensor([1, 2, 3, 4, 5])}
{23341488734: tensor([1, 2, 3, 4, 5])}
{24505608366: tensor([1, 2, 3, 4, 5])}
{394518178: tensor([1, 2, 3

In [255]:
bob._objects

{18757266557: tensor([1, 2, 3, 4, 5])}

In [203]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [204]:
x

(Wrapper)>[PointerTensor | me:54018817236 -> bob:52219483468]

In [205]:
y

tensor([1, 1, 1, 1, 1])

In [206]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:54018817236 -> bob:52219483468]
Tensor B: tensor([1, 1, 1, 1, 1])

In [207]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [208]:
x

(Wrapper)>[PointerTensor | me:40517550412 -> bob:33584040525]

In [209]:
y

(Wrapper)>[PointerTensor | me:52901748157 -> alice:26162460310]

In [199]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [4]:
from torch import nn, optim
import torch as th
import syft as sy

In [11]:
hook = sy.TorchHook(th)

In [12]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [5]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [6]:
# A Toy Model
model = nn.Linear(2,1)

In [7]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [9]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0020)
tensor(0.0015)
tensor(0.0012)
tensor(0.0009)
tensor(0.0007)
tensor(0.0005)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(0.0001)
tensor(7.8590e-05)
tensor(6.0075e-05)
tensor(4.5924e-05)
tensor(3.5108e-05)
tensor(2.6841e-05)
tensor(2.0521e-05)
tensor(1.5689e-05)
tensor(1.1996e-05)


In [13]:
bob

<VirtualWorker id:bob #objects:0>

In [14]:
alice

<VirtualWorker id:alice #objects:0>

In [15]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [16]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [17]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [24]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)
            print(iter)
            print(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [25]:
train()

0
<VirtualWorker id:bob #objects:6>
tensor(2.0736, requires_grad=True)
0
<VirtualWorker id:alice #objects:4>
tensor(0.0417, requires_grad=True)
1
<VirtualWorker id:bob #objects:6>
tensor(0.0634, requires_grad=True)
1
<VirtualWorker id:alice #objects:4>
tensor(0.0357, requires_grad=True)
2
<VirtualWorker id:bob #objects:6>
tensor(0.0315, requires_grad=True)
2
<VirtualWorker id:alice #objects:4>
tensor(0.0237, requires_grad=True)
3
<VirtualWorker id:bob #objects:6>
tensor(0.0206, requires_grad=True)
3
<VirtualWorker id:alice #objects:4>
tensor(0.0158, requires_grad=True)
4
<VirtualWorker id:bob #objects:6>
tensor(0.0138, requires_grad=True)
4
<VirtualWorker id:alice #objects:4>
tensor(0.0108, requires_grad=True)
5
<VirtualWorker id:bob #objects:6>
tensor(0.0094, requires_grad=True)
5
<VirtualWorker id:alice #objects:4>
tensor(0.0075, requires_grad=True)
6
<VirtualWorker id:bob #objects:6>
tensor(0.0065, requires_grad=True)
6
<VirtualWorker id:alice #objects:4>
tensor(0.0054, requires_gra

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [27]:
bob.clear_objects()


<VirtualWorker id:bob #objects:0>

In [28]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [29]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [30]:
x = x.send(alice)

In [31]:
bob._objects

{2494493128: tensor([1, 2, 3, 4, 5])}

In [32]:
alice._objects

{85416304050: (Wrapper)>[PointerTensor | alice:85416304050 -> bob:2494493128]}

In [33]:
y = x + x

In [34]:
y

(Wrapper)>[PointerTensor | me:77208791303 -> alice:73175927670]

In [35]:
bob._objects

{2494493128: tensor([1, 2, 3, 4, 5]),
 46526040717: tensor([ 2,  4,  6,  8, 10])}

In [36]:
alice._objects

{85416304050: (Wrapper)>[PointerTensor | alice:85416304050 -> bob:2494493128],
 73175927670: (Wrapper)>[PointerTensor | alice:73175927670 -> bob:46526040717]}

In [37]:
jon = sy.VirtualWorker(hook, id="jon")

In [38]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [39]:
bob._objects

{59490013608: tensor([1, 2, 3, 4, 5])}

In [40]:
alice._objects

{97505042588: (Wrapper)>[PointerTensor | alice:97505042588 -> bob:59490013608]}

In [41]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:97505042588 -> bob:59490013608]

In [42]:
bob._objects

{59490013608: tensor([1, 2, 3, 4, 5])}

In [43]:
alice._objects

{}

In [44]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [45]:
bob._objects

{}

In [46]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [47]:
bob._objects

{26133374431: tensor([1, 2, 3, 4, 5])}

In [48]:
alice._objects

{89374333351: (Wrapper)>[PointerTensor | alice:89374333351 -> bob:26133374431]}

In [49]:
del x

In [50]:
bob._objects

{}

In [51]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [52]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [53]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [54]:
bob._objects

{10960223670: tensor([1, 2, 3, 4, 5])}

In [55]:
alice._objects

{}

In [56]:
x.move(alice)

(Wrapper)>[PointerTensor | me:72909658614 -> alice:72909658614]

In [57]:
bob._objects

{}

In [58]:
alice._objects

{72909658614: tensor([1, 2, 3, 4, 5])}

In [59]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [60]:
bob._objects

{26965868044: tensor([1, 2, 3, 4, 5])}

In [61]:
alice._objects

{72909658614: tensor([1, 2, 3, 4, 5]),
 93772937700: (Wrapper)>[PointerTensor | alice:93772937700 -> bob:26965868044]}

In [62]:
x.remote_get()

(Wrapper)>[PointerTensor | me:39924195124 -> alice:93772937700]

In [63]:
bob._objects

{}

In [64]:
alice._objects

{72909658614: tensor([1, 2, 3, 4, 5]), 93772937700: tensor([1, 2, 3, 4, 5])}

In [65]:
x.move(bob)

(Wrapper)>[PointerTensor | me:39924195124 -> bob:39924195124]

In [66]:
x

(Wrapper)>[PointerTensor | me:39924195124 -> bob:39924195124]

In [67]:
bob._objects

{39924195124: tensor([1, 2, 3, 4, 5])}

In [68]:
alice._objects

# This is a garbage collection issue. 
# Alice shouldn't have this data

{72909658614: tensor([1, 2, 3, 4, 5])}